In [1]:
import os
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from typing import Literal
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from openai import OpenAI


In [2]:
cd ../

d:\Project\Python\Automatic Anlalyz\dagpt


d:\Project\Python\Automatic Anlalyz\dagpt\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
cd dagpt

[WinError 2] The system cannot find the file specified: 'dagpt'
d:\Project\Python\Automatic Anlalyz\dagpt


d:\Project\Python\Automatic Anlalyz\dagpt\.venv\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
# Ensure environment variables are loaded
load_dotenv()

True

In [5]:
import sys
sys.path.append(r'D:\Project\Python\Automatic Anlalyz\dagpt\src')
from src.Logger.base import BaseLogger
from src.models.llms import load_llm


# Check if GEMINI_KEY_API is set
if 'GEMINI_KEY_API' not in os.environ:
	raise KeyError('GEMINI_KEY_API not found in environment variables')

In [6]:
MODEL_NAME = "gemini-1.5-pro"
llm = load_llm(model_name=MODEL_NAME)
llm

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [7]:
response = llm.generate_content("Can you give me more details about crypto?")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Crypto, short for cryptocurrency, is a digital or virtual currency designed to work as a medium of exchange. It uses cryptography to secure and verify transactions as well as to control the creation of new units of a particular cryptocurrency.  Here's a breakdown of key aspects:\n\n**Key Concepts:**\n\n* **Decentralization:**  Unlike traditional currencies issued by central banks, cryptocurrencies operate on a decentralized network, typically a blockchain. This means no single entity controls it, making it resistant to censorship and government manipulation.\n* **Blockchain:** A distributed ledger technology that records all transactions across multiple computers. This makes it transparent and virtually tamper-proof.\n* **Cryptography:**  Uses complex mathem

In [11]:
df = pd.read_csv(r".\data\raw\sample_data.csv")


da_agent = create_pandas_dataframe_agent(
        llm=ChatGoogleGenerativeAI(
        # model="gemini-1.5-pro",
        model= MODEL_NAME,
        temperature=0,
        google_api_key=os.getenv("GEMINI_KEY_API")
    ),
    df=df,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    verbose=True,
    return_intermediate_steps=True,
)
da_agent

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\DELL\AppData\Local\Temp\ipykernel_2764\191557769.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv(".\data\raw\sample_data.csv")
C:\Users\DELL\AppData\Local\Temp\ipykernel_2764\191557769.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv(".\data\raw\sample_data.csv")


OSError: [Errno 22] Invalid argument: '.\\data\raw\\sample_data.csv'